In [1]:
import pandas as pd
import numpy as np
np.seterr(all='raise')
%load_ext line_profiler

In [2]:
df = pd.read_pickle("../results/mibig/features/1.pkl")

In [3]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.exceptions import ConvergenceWarning
from scipy.stats import anderson
import warnings

def get_clusters(km):
    "get clusters array from fitted k-means object"
    return np.array([np.where(km.labels_ == i) for i in range(len(km.cluster_centers_))])


def expand_clusters(data, clusters, centers):
    "for every cluster, try split into two"
    new_centers = []
    for i, cluster in enumerate(clusters):
        if len(centers) == 1:
            data_cluster = data
        else:
            data_cluster = data[tuple(cluster)]
            
        if len(data_cluster) > 2:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                km = KMeans(n_clusters=2, init="k-means++", n_init=1, precompute_distances=True, tol=0.025, algorithm="full").fit(data_cluster)            
                if len(set(km.labels_)) == 2 and not accept_test(data_cluster, km.cluster_centers_):
                    new_centers.extend(km.cluster_centers_)
                    continue
        new_centers.append(centers[i])
        
    return np.array(new_centers)


def accept_test(data, centers):
    "perform Anderson-Darling test"
    assert(len(centers) == 2)
    v = np.subtract(centers[0], centers[1])
    square_norm = np.sum(np.multiply(v, v))
    points = np.divide(np.sum(np.multiply(data, v), axis=1), square_norm)

    estimation, critical, _ = anderson(points, dist='norm')
    return estimation < critical[-1]


def gmeans(data: np.array):
    # perform initial clustering with k=1
    km = KMeans(n_clusters=1, init="random", n_init=1, precompute_distances=False, algorithm="full").fit(data)
    #km = MiniBatchKMeans(n_clusters=1, init="random", n_init=1).fit(data)
    clusters, centers = get_clusters(km), km.cluster_centers_
    while True:
        new_centers = expand_clusters(data, clusters, centers)
        if len(centers) == len(new_centers): # convergence
            print(str(len(new_centers)))
            break
        # re-perform k-means, with existing centers
        km = KMeans(n_clusters=len(new_centers), init=new_centers, n_init=1, precompute_distances=True, tol=0.025, algorithm="full").fit(data)
        #km = MiniBatchKMeans(n_clusters=len(new_centers), init=new_centers, batch_size=len(new_centers)*3, tol=0.025).fit(data)
        clusters, centers = get_clusters(km), km.cluster_centers_

    return clusters, centers

In [4]:
sample = df.sample(500).values

In [ ]:
%time cl, cs = gmeans(sample)

In [ ]:
from pyclustering.cluster.gmeans import gmeans as pygmeans

gmeans_py = pygmeans(sample)
%time gmeans_py.process()
len(gmeans_py.get_centers())

gmeans_py2 = pygmeans(sample)
%time gmeans_py2.process()
len(gmeans_py2.get_centers())

In [ ]:
from sklearn.metrics import adjusted_rand_score

labels_gm = [None for i in range(len(sample))]
for ci, c in enumerate(cl):
    for i in c[0]:
        labels_gm[i] = ci
        
labels_pygm = [None for i in range(len(sample))]
for ci, c in enumerate(gmeans_py.get_clusters()):
    for i in c:
        labels_pygm[i] = ci
        
print(adjusted_rand_score(labels_gm, labels_pygm))

labels_pygm2 = [None for i in range(len(sample))]
for ci, c in enumerate(gmeans_py2.get_clusters()):
    for i in c:
        labels_pygm2[i] = ci
        
print(adjusted_rand_score(labels_pygm2, labels_pygm))

In [12]:
from sklearn.cluster import MiniBatchKMeans

%time mb = MiniBatchKMeans(n_clusters=5, init="k-means++").fit(sample)

CPU times: user 1.85 s, sys: 287 ms, total: 2.14 s
Wall time: 1.13 s


In [13]:
from sklearn.cluster import KMeans

%time km = KMeans(n_clusters=5, init="k-means++").fit(sample)

CPU times: user 5.22 s, sys: 314 ms, total: 5.53 s
Wall time: 3.9 s


In [14]:
from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(mb.labels_, km.labels_)

0.7558637845625084